In [1]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [2]:
source_dir = './BOEs'
target_dir = './BOEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BOE de hoy

In [3]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210224
dia = 24
mes = 02
año = 2021
24/02/2021


In [4]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [5]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210224'

In [6]:
url = URL_XML_resumen
r = requests.get(url)

In [7]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [8]:
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [9]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [10]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [11]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']


In [12]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [13]:
tabla_resumen = pd.DataFrame()

In [17]:
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales") or contains(@nombre, "III. Otras disposiciones")]'):
#for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):

    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


['I. Disposiciones generales']
['III. Otras disposiciones']


In [18]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Tratados internacionales],[Instrumento de ratificación del Acuerdo de as...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2840],[I. Disposiciones generales]
1,[CORTES GENERALES],[Medidas urgentes],"[Resolución de 18 de febrero de 2021, del Cong...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2841],[I. Disposiciones generales]
2,[CORTES GENERALES],[Medidas urgentes],"[Resolución de 18 de febrero de 2021, del Cong...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2842],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Tratados internacionales],[Enmiendas de 2018 al Código internacional de ...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2843],[I. Disposiciones generales]
4,[MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL],[Organización],"[Real Decreto 117/2021, de 23 de febrero, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2844],[I. Disposiciones generales]
5,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Protección de los animales],"[Real Decreto 118/2021, de 23 de febrero, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2845],[I. Disposiciones generales]
6,[MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN ...,[Funcionarios de administración local con habi...,"[Orden TFP/153/2021, de 16 de febrero, por la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2846],[I. Disposiciones generales]
7,[COMUNIDAD AUTÓNOMA DE CATALUÑA],[Medidas sociales y económicas],[Corrección de errores del Decreto-ley 51/2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2847],[I. Disposiciones generales]
8,[COMUNIDAD AUTÓNOMA DE CASTILLA-LA MANCHA],[Sanidad],"[Ley 8/2020, de 16 de octubre, por la que se c...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2848],[I. Disposiciones generales]
9,[COMUNIDAD AUTÓNOMA DE CASTILLA-LA MANCHA],[Patrimonio],"[Ley 9/2020, de 6 de noviembre, de Patrimonio ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2849],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [19]:
for item_URL in tabla_resumen['Item_URL_XML']:
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2840']
<re.Match object; span=(0, 15), match='BOE-A-2021-2840'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2841']
<re.Match object; span=(0, 15), match='BOE-A-2021-2841'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2842']
<re.Match object; span=(0, 15), match='BOE-A-2021-2842'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2843']
<re.Match object; span=(0, 15), match='BOE-A-2021-2843'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2844']
<re.Match object; span=(0, 15), match='BOE-A-2021-2844'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2845']
<re.Match object; span=(0, 15), match='BOE-A-2021-2845'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2846']
<re.Match object; span=(0, 15), match='BOE-A-2021-2846'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2847']
<re.Match object; span=(0, 15), match='BOE-A-2021-2847'>
['https://www.boe.es/diario_boe/

# Importa de nuevo los XML generados

In [20]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2021-2898.xml', './BOEs/BOE-A-2021-2888.xml', './BOEs/BOE-A-2021-2889.xml', './BOEs/BOE-A-2021-2848.xml', './BOEs/BOE-A-2021-2849.xml', './BOEs/BOE-A-2021-2887.xml', './BOEs/BOE-A-2021-2893.xml', './BOEs/BOE-A-2021-2844.xml', './BOEs/BOE-A-2021-2845.xml', './BOEs/BOE-A-2021-2892.xml', './BOEs/BOE-A-2021-2886.xml', './BOEs/BOE-A-2021-2890.xml', './BOEs/BOE-A-2021-2884.xml', './BOEs/BOE-A-2021-2847.xml', './BOEs/BOE-A-2021-2846.xml', './BOEs/BOE-A-2021-2885.xml', './BOEs/BOE-A-2021-2891.xml', './BOEs/BOE-A-2021-2895.xml', './BOEs/BOE-A-2021-2881.xml', './BOEs/BOE-A-2021-2842.xml', './BOEs/BOE-A-2021-2843.xml', './BOEs/BOE-A-2021-2894.xml', './BOEs/BOE-A-2021-2882.xml', './BOEs/BOE-A-2021-2896.xml', './BOEs/BOE-A-2021-2841.xml', './BOEs/BOE-A-2021-2840.xml', './BOEs/BOE-A-2021-2897.xml', './BOEs/BOE-A-2021-2883.xml']


# Genera DF con datos Análisis de cada XML

In [21]:
tabla_analisis = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]

    Item_Id = raiz.xpath(".//identificador/text()")
    Item_name = raiz.xpath('.//titulo/text()') 

    Fecha_Publicacion = datetime.strptime(str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2],"%Y%m%d")
    #Fecha_Publicacion = str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2]
 
    tabla_analisis = tabla_analisis.append({'Item_id': Item_Id,
                                            'Item_Name' : Item_name,
                                            'Fecha_publicacion' : Fecha_Publicacion,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


In [22]:
tabla_analisis.sort_values('Item_id')

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto
25,"[Relaciones internacionales, Unión Europea]",2021-02-24,[Instrumento de ratificación del Acuerdo de as...,[BOE-A-2021-2840],"[Acuerdo de Asociación CE, Acuerdos internacio...",[],[],[]
24,"[Administración electrónica, Asuntos sociales,...",2021-02-24,"[Resolución de 18 de febrero de 2021, del Cong...",[BOE-A-2021-2841],"[Administración electrónica, Arrendamientos ur...",[],[],[]
19,"[Asuntos sociales, Derecho Administrativo, Org...",2021-02-24,"[Resolución de 18 de febrero de 2021, del Cong...",[BOE-A-2021-2842],[Accidentes de trabajo y enfermedades profesio...,[BOE-A-2021-1529],[PUBLICA Acuerdo de convalidación],"[del Real Decreto-ley 3/2021, de 2 de febrero]"
20,[],2021-02-24,[Enmiendas de 2018 al Código internacional de ...,[BOE-A-2021-2843],[],[],[],[]
7,[Organización de la Administración],2021-02-24,"[Real Decreto 117/2021, de 23 de febrero, por ...",[BOE-A-2021-2844],"[Consejo de Fomento de la Economía Social, Min...","[BOE-A-2001-5061, BOE-A-2011-5708, BOE-A-2007-...","[MODIFICA, DE CONFORMIDAD con, DE CONFORMIDAD ...","[los arts. 1, 2, 3, 4, 8, 12 y 13, la disposic..."
8,"[Ganadería y animales, Tecnología e investigac...",2021-02-24,"[Real Decreto 118/2021, de 23 de febrero, por ...",[BOE-A-2021-2845],"[Acceso a la información, Animales, Comisión E...",[BOE-A-2013-1337],[MODIFICA],"[los arts. 34.2.e), 36, 41.4 del Real Decreto ..."
14,[Función Pública],2021-02-24,"[Orden TFP/153/2021, de 16 de febrero, por la ...",[BOE-A-2021-2846],"[Funcionarios de la Administración Local, Opos...",[],[],[]
13,"[Asuntos sociales, Sanidad]",2021-02-24,[Corrección de errores del Decreto-ley 51/2020...,[BOE-A-2021-2847],"[Ancianos, Asistencia sanitaria, Asistencia so...","[BOE-A-2021-1479, BOE-A-2020-15840]","[CORRECCIÓN de errores, CORRIGE errores]","[del Decreto-ley 51/2020, de 15 de diciembre, ..."
3,[],2021-02-24,"[Ley 8/2020, de 16 de octubre, por la que se c...",[BOE-A-2021-2848],"[Castilla La Mancha, Consumidores y usuarios, ...","[BOE-A-1982-20820, BOE-A-2020-3692]","[DE CONFORMIDAD con, CITA]",[el Estatuto aprobado por por Ley Orgánica 9/1...
4,[],2021-02-24,"[Ley 9/2020, de 6 de noviembre, de Patrimonio ...",[BOE-A-2021-2849],"[Castilla La Mancha, Organización de las Comun...","[BOE-A-1985-26400, ---, BOE-A-2006-11292, BOE-...","[DEROGA, DEROGA, MODIFICA, DE CONFORMIDAD con]","[la Ley 6/1985, de 13 de noviembre, en la form..."


In [23]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
tabla_analisis['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Tags'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Match_ASECORP_BBDD'] = [[] for i in range(len(tabla_analisis))]


In [24]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [25]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
for i, row in tabla_analisis.iterrows():
    for item_list in range(len(row['Referencias'])): 
        row['Referencias_completas'].append(row['Referencias_palabra'][item_list] + ' ' + row['Referencias_texto'][item_list])
        tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)
    print(row['Referencias_completas'])

[]
[]
[]
['DE CONFORMIDAD con el Estatuto aprobado por por Ley Orgánica 9/1982, de 10 de agosto', 'CITA Real Decreto 463/2020, de 14 de marzo']
['DEROGA la Ley 6/1985, de 13 de noviembre', 'DEROGA en la forma indicada el Decreto 104/1986, de 23 de septiembre (DOCM núm. 42, de octubre de 1986)', 'MODIFICA el art. 5 de la Ley 1/2006, de 23 de marzo', 'DE CONFORMIDAD con el Estatuto aprobado por Ley Orgánica 9/1982, de 10 de agosto']
[]
[]
['MODIFICA los arts. 1, 2, 3, 4, 8, 12 y 13, la disposición adicional única y la disposición final 1 y AÑADE la disposición adicional 2 al Real Decreto 219/2001, de 2 de marzo', 'DE CONFORMIDAD con el art. 13 de la Ley 5/2011, de 29 de marzo', 'DE CONFORMIDAD con la Ley Orgánica 3/2007, de 22 de marzo', 'CITA Ley 27/1999, de 16 de julio']
['MODIFICA los arts. 34.2.e), 36, 41.4 del Real Decreto 53/2013, de 1 de febrero']
[]
[]
[]
[]
['CORRECCIÓN de errores del Decreto-ley 51/2020, de 15 de diciembre', 'CORRIGE errores del Decreto-ley 39/2020, de 3 de nov

In [26]:
# Elimina Tags duplicados
for i, row in tabla_analisis.iterrows():
    tabla_analisis['Tags'][i] = list(set(tabla_analisis['Tags'][i]))
    print(tabla_analisis['Tags'][i])

[]
[]
[]
['Real Decreto 463/2020']
['Ley 1/2006', 'Ley 6/1985']
[]
[]
['Ley 27/1999', 'Ley 5/2011', 'Real Decreto 219/2001']
['Real Decreto 53/2013']
[]
[]
[]
[]
['ley 39/2020', 'ley 51/2020']
[]
[]
[]
[]
[]
['Real Decreto-ley 3/2021']
[]
[]
[]
[]
[]
[]
[]
[]


In [27]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[Cultura y ocio],2021-02-24,"[Decreto 102/2020, de 9 de diciembre, por el q...",[BOE-A-2021-2898],[],[],[],[],[],[],[]
1,[],2021-02-24,"[Resolución de 2 de febrero de 2021, de la Sec...",[BOE-A-2021-2888],[],[],[],[],[],[],[]
2,[],2021-02-24,"[Resolución de 2 de febrero de 2021, de la Sec...",[BOE-A-2021-2889],[],[],[],[],[],[],[]
3,[],2021-02-24,"[Ley 8/2020, de 16 de octubre, por la que se c...",[BOE-A-2021-2848],"[Castilla La Mancha, Consumidores y usuarios, ...","[BOE-A-1982-20820, BOE-A-2020-3692]","[DE CONFORMIDAD con, CITA]",[el Estatuto aprobado por por Ley Orgánica 9/1...,[DE CONFORMIDAD con el Estatuto aprobado por p...,[Real Decreto 463/2020],[]
4,[],2021-02-24,"[Ley 9/2020, de 6 de noviembre, de Patrimonio ...",[BOE-A-2021-2849],"[Castilla La Mancha, Organización de las Comun...","[BOE-A-1985-26400, ---, BOE-A-2006-11292, BOE-...","[DEROGA, DEROGA, MODIFICA, DE CONFORMIDAD con]","[la Ley 6/1985, de 13 de noviembre, en la form...","[DEROGA la Ley 6/1985, de 13 de noviembre, DER...","[Ley 1/2006, Ley 6/1985]",[]
5,[],2021-02-24,"[Resolución de 15 de febrero de 2021, de la En...",[BOE-A-2021-2887],[],[],[],[],[],[],[]
6,[Sistema financiero],2021-02-24,"[Resolución de 11 de febrero de 2021, de la Di...",[BOE-A-2021-2893],[],[],[],[],[],[],[]
7,[Organización de la Administración],2021-02-24,"[Real Decreto 117/2021, de 23 de febrero, por ...",[BOE-A-2021-2844],"[Consejo de Fomento de la Economía Social, Min...","[BOE-A-2001-5061, BOE-A-2011-5708, BOE-A-2007-...","[MODIFICA, DE CONFORMIDAD con, DE CONFORMIDAD ...","[los arts. 1, 2, 3, 4, 8, 12 y 13, la disposic...","[MODIFICA los arts. 1, 2, 3, 4, 8, 12 y 13, la...","[Ley 27/1999, Ley 5/2011, Real Decreto 219/2001]",[]
8,"[Ganadería y animales, Tecnología e investigac...",2021-02-24,"[Real Decreto 118/2021, de 23 de febrero, por ...",[BOE-A-2021-2845],"[Acceso a la información, Animales, Comisión E...",[BOE-A-2013-1337],[MODIFICA],"[los arts. 34.2.e), 36, 41.4 del Real Decreto ...","[MODIFICA los arts. 34.2.e), 36, 41.4 del Real...",[Real Decreto 53/2013],[]
9,[Deporte],2021-02-24,"[Resolución de 12 de febrero de 2021, de la Pr...",[BOE-A-2021-2892],"[Asociaciones deportivas, Federación Española ...",[],[],[],[],[],[]


In [28]:
texto = ''
for refs in tabla_analisis['Referencias_completas']:
    for ref in refs:
        texto = texto + ref + ' '
        print(ref)

DE CONFORMIDAD con el Estatuto aprobado por por Ley Orgánica 9/1982, de 10 de agosto
CITA Real Decreto 463/2020, de 14 de marzo
DEROGA la Ley 6/1985, de 13 de noviembre
DEROGA en la forma indicada el Decreto 104/1986, de 23 de septiembre (DOCM núm. 42, de octubre de 1986)
MODIFICA el art. 5 de la Ley 1/2006, de 23 de marzo
DE CONFORMIDAD con el Estatuto aprobado por Ley Orgánica 9/1982, de 10 de agosto
MODIFICA los arts. 1, 2, 3, 4, 8, 12 y 13, la disposición adicional única y la disposición final 1 y AÑADE la disposición adicional 2 al Real Decreto 219/2001, de 2 de marzo
DE CONFORMIDAD con el art. 13 de la Ley 5/2011, de 29 de marzo
DE CONFORMIDAD con la Ley Orgánica 3/2007, de 22 de marzo
CITA Ley 27/1999, de 16 de julio
MODIFICA los arts. 34.2.e), 36, 41.4 del Real Decreto 53/2013, de 1 de febrero
CORRECCIÓN de errores del Decreto-ley 51/2020, de 15 de diciembre
CORRIGE errores del Decreto-ley 39/2020, de 3 de noviembre
PRORROGA lo indicado del Acuerdo de 14 de febrero de 2019
PUBL

In [29]:
texto = ''
for i, row in tabla_analisis.iterrows():
    #unique_id = i
    print('\n' + str(row['Item_id']) + str(row['Item_Name']) +'\n')

    antecedente = 1
    for ref in row['Referencias_completas']:
        texto = texto + ref + ' '
        print('\t' + 'Antecedente ' + str(antecedente) + ': ' + ref + ' ' + str(re.findall('|'.join(pattern), ref, flags=re.IGNORECASE)))
        antecedente += 1


['BOE-A-2021-2898']['Decreto 102/2020, de 9 de diciembre, por el que se declara bien de interés cultural, con la categoría de monumento, la Iglesia parroquial Nuestra Señora de la Asunción y su entorno de protección en Navarrete.']


['BOE-A-2021-2888']['Resolución de 2 de febrero de 2021, de la Secretaría General de Coordinación Territorial, por la que se publica el Acuerdo de la Subcomisión de Seguimiento Normativo, Prevención y Solución de Conflictos de la Comisión Bilateral Generalitat-Estado, en relación con el Decreto-ley 37/2020, de 3 de noviembre, de refuerzo de la protección del derecho a la vivienda ante los efectos de la pandemia de la COVID-19.']


['BOE-A-2021-2889']['Resolución de 2 de febrero de 2021, de la Secretaría General de Coordinación Territorial, por la que se publica el Acuerdo de la Subcomisión de Seguimiento Normativo, Prevención y Solución de Conflictos de la Comisión Bilateral Generalitat-Estado, en relación con el Decreto-ley 34/2020, de 20 de octubre, de 

In [30]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = re.findall('|'.join(pattern), texto, flags=re.IGNORECASE)

In [31]:
print(regex_result)

['Real Decreto 463/2020', 'Ley 6/1985', 'Ley 1/2006', 'Real Decreto 219/2001', 'Ley 5/2011', 'Ley 27/1999', 'Real Decreto 53/2013', 'ley 51/2020', 'ley 39/2020', 'Real Decreto-ley 3/2021']


In [32]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [33]:
boletin_flat_list

['Real Decreto 463/2020',
 'Ley 6/1985',
 'ley 51/2020',
 'Ley 27/1999',
 'Ley 5/2011',
 'Ley 1/2006',
 'ley 39/2020',
 'Real Decreto 219/2001',
 'Real Decreto-ley 3/2021',
 'Real Decreto 53/2013']

## Importa BBDD ASECORP

In [34]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [35]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [36]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [37]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [38]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [39]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [40]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [41]:
# titulo

In [42]:
len(titulo)

2959

In [43]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [44]:
boletin[0:25]

['[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [45]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [46]:
n_vacios

276

In [47]:
vacios[0:25]

['Acuerdo Multilateral RID 1/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Mercancías Peligrosas por Ferrocarril (RID), relativo a los certificados de consejeros de seguridad de conformidad con el punto 1.8.3.7 del RID, hecho en Madrid el 25 de marzo de 2020',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Acuerdo Multilateral RID 2/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Merc

In [48]:
boletin[0:25]

[['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268'],
 ['Acuerdo Multilateral M-271'],
 ['Acuerdo Multilateral M-273'],
 ['Acuerdo Multilateral M-276'],
 ['Acuerdo Multilateral M-291'],
 ['Acuerdo Multilateral M-292']]

In [49]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [50]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268',
 'Acuerdo Multilateral M-271',
 'Acuerdo Multilateral M-273',
 'Acuerdo Multilateral M-276',
 'Acuerdo Multilateral M-291',
 'Acuerdo Multilateral M-292']

In [51]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [52]:
boletin_ASECORP_flat_list[0:25]

['Real Decreto 578/2014',
 'Real Decreto Legislativo 8/2015',
 'Acuerdo Multilateral M-236',
 'Real Decreto 1477/1990',
 'Orden INT/2850/2011',
 'Real Decreto 942/2005',
 'Resolución de 4 de noviembre de 1982',
 'Orden PRE/1131/2011',
 'Real Decreto-ley 6/2009',
 'Orden ITC/1251/2009',
 'Real Decreto 1598/2011',
 'Resolución de 10 de enero de 2020',
 'Acuerdo Multilateral M-259',
 'Orden FOM/534/2014',
 'Real Decreto 1330/2012',
 'Real decreto 1841/1997',
 'Ley 30/2007',
 'Acuerdo Multilateral M-157',
 'Real Decreto 294/2016',
 'Orden de 30 de abril de 2020',
 'Ley 38/1994',
 'Resolución de 31 de agosto de 2015',
 'Orden MAM/638/2002',
 'Resolución de 11 de febrero de 2014',
 'Orden de 28 de octubre de 1992']

In [53]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Real Decreto 463/2020'}

In [54]:
#tabla_analisis['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in tabla_analisis['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Real Decreto 463/2020'}
{'Real Decreto 463/2020'}


In [55]:
for i, row_to_compare in tabla_analisis.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            tabla_analisis['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Real Decreto 463/2020'} ORD INT/262/2020 ESP
{'Real Decreto 463/2020'} ORD INT/284/2020 ESP


In [56]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[Cultura y ocio],2021-02-24,"[Decreto 102/2020, de 9 de diciembre, por el q...",[BOE-A-2021-2898],[],[],[],[],[],[],[]
1,[],2021-02-24,"[Resolución de 2 de febrero de 2021, de la Sec...",[BOE-A-2021-2888],[],[],[],[],[],[],[]
2,[],2021-02-24,"[Resolución de 2 de febrero de 2021, de la Sec...",[BOE-A-2021-2889],[],[],[],[],[],[],[]
3,[],2021-02-24,"[Ley 8/2020, de 16 de octubre, por la que se c...",[BOE-A-2021-2848],"[Castilla La Mancha, Consumidores y usuarios, ...","[BOE-A-1982-20820, BOE-A-2020-3692]","[DE CONFORMIDAD con, CITA]",[el Estatuto aprobado por por Ley Orgánica 9/1...,[DE CONFORMIDAD con el Estatuto aprobado por p...,[Real Decreto 463/2020],"[ORD INT/262/2020 ESP, ORD INT/284/2020 ESP]"
4,[],2021-02-24,"[Ley 9/2020, de 6 de noviembre, de Patrimonio ...",[BOE-A-2021-2849],"[Castilla La Mancha, Organización de las Comun...","[BOE-A-1985-26400, ---, BOE-A-2006-11292, BOE-...","[DEROGA, DEROGA, MODIFICA, DE CONFORMIDAD con]","[la Ley 6/1985, de 13 de noviembre, en la form...","[DEROGA la Ley 6/1985, de 13 de noviembre, DER...","[Ley 1/2006, Ley 6/1985]",[]
5,[],2021-02-24,"[Resolución de 15 de febrero de 2021, de la En...",[BOE-A-2021-2887],[],[],[],[],[],[],[]
6,[Sistema financiero],2021-02-24,"[Resolución de 11 de febrero de 2021, de la Di...",[BOE-A-2021-2893],[],[],[],[],[],[],[]
7,[Organización de la Administración],2021-02-24,"[Real Decreto 117/2021, de 23 de febrero, por ...",[BOE-A-2021-2844],"[Consejo de Fomento de la Economía Social, Min...","[BOE-A-2001-5061, BOE-A-2011-5708, BOE-A-2007-...","[MODIFICA, DE CONFORMIDAD con, DE CONFORMIDAD ...","[los arts. 1, 2, 3, 4, 8, 12 y 13, la disposic...","[MODIFICA los arts. 1, 2, 3, 4, 8, 12 y 13, la...","[Ley 27/1999, Ley 5/2011, Real Decreto 219/2001]",[]
8,"[Ganadería y animales, Tecnología e investigac...",2021-02-24,"[Real Decreto 118/2021, de 23 de febrero, por ...",[BOE-A-2021-2845],"[Acceso a la información, Animales, Comisión E...",[BOE-A-2013-1337],[MODIFICA],"[los arts. 34.2.e), 36, 41.4 del Real Decreto ...","[MODIFICA los arts. 34.2.e), 36, 41.4 del Real...",[Real Decreto 53/2013],[]
9,[Deporte],2021-02-24,"[Resolución de 12 de febrero de 2021, de la Pr...",[BOE-A-2021-2892],"[Asociaciones deportivas, Federación Española ...",[],[],[],[],[],[]


In [57]:
tabla_analisis['Match_ASECORP_BBDD'][1]

[]

In [58]:
tabla_analisis['Tags'][1]

[]

# Genera Fichero EXCEL de resultados

In [59]:
## Cambia orden de columnas y elimina las no necesarias  
tabla_analisis_final = tabla_analisis[['Item_id','Item_Name','Fecha_publicacion','Materias','Alertas','Referencias','Referencias_completas','Tags','Match_ASECORP_BBDD']]

In [60]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [61]:
## Aplica función de conversión de listas a strings
tabla_analisis_final = tabla_analisis_final.apply(lambda x: [list2Str(i) for i in x])

In [62]:
  tabla_analisis_final[0:10]

,Item_id,Item_Name,Fecha_publicacion,Materias,Alertas,Referencias,Referencias_completas,Tags,Match_ASECORP_BBDD
0,BOE-A-2021-2898,"Decreto 102/2020, de 9 de diciembre, por el qu...",2021-02-24,,Cultura y ocio,,,,
1,BOE-A-2021-2888,"Resolución de 2 de febrero de 2021, de la Secr...",2021-02-24,,,,,,
2,BOE-A-2021-2889,"Resolución de 2 de febrero de 2021, de la Secr...",2021-02-24,,,,,,
3,BOE-A-2021-2848,"Ley 8/2020, de 16 de octubre, por la que se cr...",2021-02-24,"Castilla La Mancha, Consumidores y usuarios, E...",,"BOE-A-1982-20820, BOE-A-2020-3692",DE CONFORMIDAD con el Estatuto aprobado por po...,Real Decreto 463/2020,"ORD INT/262/2020 ESP, ORD INT/284/2020 ESP"
4,BOE-A-2021-2849,"Ley 9/2020, de 6 de noviembre, de Patrimonio d...",2021-02-24,"Castilla La Mancha, Organización de las Comuni...",,"BOE-A-1985-26400, ---, BOE-A-2006-11292, BOE-A...","DEROGA la Ley 6/1985, de 13 de noviembre, DERO...","Ley 1/2006, Ley 6/1985",
5,BOE-A-2021-2887,"Resolución de 15 de febrero de 2021, de la Ent...",2021-02-24,,,,,,
6,BOE-A-2021-2893,"Resolución de 11 de febrero de 2021, de la Dir...",2021-02-24,,Sistema financiero,,,,
7,BOE-A-2021-2844,"Real Decreto 117/2021, de 23 de febrero, por e...",2021-02-24,"Consejo de Fomento de la Economía Social, Mini...",Organización de la Administración,"BOE-A-2001-5061, BOE-A-2011-5708, BOE-A-2007-6...","MODIFICA los arts. 1, 2, 3, 4, 8, 12 y 13, la ...","Ley 27/1999, Ley 5/2011, Real Decreto 219/2001",
8,BOE-A-2021-2845,"Real Decreto 118/2021, de 23 de febrero, por e...",2021-02-24,"Acceso a la información, Animales, Comisión Eu...","Ganadería y animales, Tecnología e investigaci...",BOE-A-2013-1337,"MODIFICA los arts. 34.2.e), 36, 41.4 del Real ...",Real Decreto 53/2013,
9,BOE-A-2021-2892,"Resolución de 12 de febrero de 2021, de la Pre...",2021-02-24,"Asociaciones deportivas, Federación Española d...",Deporte,,,,


In [63]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

tabla_analisis_final_CSV = tabla_analisis_final

In [64]:
# Compone y genera enlace a PDF del BOE correspondiente
tabla_analisis_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"https://www.boe.es/boe/dias/' \
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%Y')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%m')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%d')) + '/'\
                                                        + 'pdfs/'\
                                                        + tabla_analisis_final_CSV['Item_id'] + '.pdf";'\
                                                        + '"' + tabla_analisis_final_CSV['Item_id'] + '")'

In [65]:
print(tabla_analisis_final_CSV['Item_id'][0])

=HIPERVINCULO("https://www.boe.es/boe/dias/2021/02/24/pdfs/BOE-A-2021-2898.pdf";"BOE-A-2021-2898")


In [66]:
# tabla_analisis_final_CSV.to_excel("./ASECORP/Resultados_Matching.xlsx", sheet_name='MATCHING_BOE') 

In [67]:
tabla_analisis_final_CSV.to_csv("./ASECORP/Resultados_Matching_BOE_" + today.strftime("%Y%m%d") + ".csv", index=False) 

# ------------------------------------------------------------------------

# DOGC

In [68]:
#URL_HTML_resumen = "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [91]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [92]:
r.html.links

{'/ca/inici/index.html',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/inici/',
 '/es/inici/resultats/index.html?orderBy=3&page=1&typeSearch=1&advanced=true&current=true&title=false&numResultsByPage=50&publicationDateInitial=18/11/2020&value=%22anunci%20d%27informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22sotmet%20a%20informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22sotmeten%20a%20informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22sobre%20informaci%C3%B3%20p%C3%BAblica%22%20OR%20%22aprovaci%C3%B3%20inicial%22%20OR%20%22anuncio%20de%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22somete%20a%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22someten%20a%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22sobre%20informaci%C3%B3n%20p%C3%BAblica%22%20OR%20%22aprobaci%C3%B3n%20inicial%22',
 '/es/inici/resultats/index.html?orderBy=3&page=1&typeSearch=1&advanced=true&current=true&title=true&numResultsByPage=50&publicationDateInitial=01/01/2021&thematicDescriptor=D194',
 '/es/inici/resulta

In [103]:
#r.html.xpath('//*[@id="titol"]/span/text()')
#r.html.xpath('//*[@class="llistat_cont"]/ul/li[1]/a/text()')
h = r.html.xpath('//*[@class="llistat_cont"]/ul/li')
[ print(item) for item in h]


[]

In [99]:
r.html.xpath('//*[@id="titol"]/h3/text()')


[]

In [73]:
r.html.lxml.tag

'html'

In [74]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[]

In [115]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)
#print(html.tostring(tree))

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)

print(tree.xpath('//*[@class="llistat-item"]/span/text()'))


[]
[]


In [76]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

[]

In [77]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

[]

In [78]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

[]

In [79]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[]

In [80]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

[]


In [81]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

[]

In [82]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

NameError: name 'argumentos' is not defined

## Recoge Nombre Secciones Sumario

In [ ]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

## Recoge Nombre Secciones Sumario

In [ ]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


In [ ]:
df_secciones_sumarios

## Recoge Valores para formar URLs Secciones Sumario

In [ ]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

In [ ]:
df_URL_sumarios

In [ ]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [ ]:
df_sumarios

In [ ]:
df_sumarios['URL_Seccion'][4]

## Recoge Items en Seccion Disposiciones

In [ ]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

In [ ]:
pdf_link

# DOUE

In [187]:
# --------------------------------------------------------------------------------------
# DOUE
# --------------------------------------------------------------------------------------

URL_HTML_resumen =  "https://eur-lex.europa.eu/oj/direct-access.html?locale=es"

## carga página HTML y genera árbol

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(URL_HTML_resumen, headers=headers)
tree = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = tree.xpath('//*[@class="table table-striped table-hover table-condensed OJTable AllBordersTable"]')
#print(secciones)


In [188]:
source_dir = './DOUEs'
target_dir = './DOUEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

In [189]:
## Recoge Valores para formar URLs Secciones Sumario

df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion_name = seccion.xpath('./tbody/tr[1]/td[2]/a/text()')
    seccion_URL = seccion.xpath('./tbody/tr[1]/td[2]/a/@href')
    #seccion = re.sub('(\\r|\\n|\\t)+', '', seccion)
    print(seccion_name)
    #print(seccion_URL)


### Si no hay disposiciones hay que gestionar el error y salir del script
if len(seccion_name) == 0:
    print('No hay disposiciones')
    # exit()

for row in range(len(seccion_name)):
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion_name[row],
                                                            'Seccion_link' : 'https://eur-lex.europa.eu' + seccion_URL[row][4:]},
                                                            ignore_index=True)

['L062', 'L063']


In [190]:
if len(df_secciones_sumarios['Seccion'][0]) == 0:
    print('No hay disposiciones')

In [191]:
df_secciones_sumarios

,Seccion,Seccion_link
0,L062,https://eur-lex.europa.eu/legal-content/ES/TXT...
1,L063,https://eur-lex.europa.eu/legal-content/ES/TXT...


In [192]:
# df_secciones_sumarios = pd.DataFrame({'Seccion':['L049'], 'Seccion_link':['https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:049:TOC']})

In [193]:
[ print (row) for row in df_secciones_sumarios['Seccion_link']]

https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:062:TOC
https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:063:TOC


[None, None]

In [194]:
DOUE_sumarios = pd.DataFrame()

for URL in df_secciones_sumarios['Seccion_link']:
    
    ### Recoge Items en Seccion Legislación

    #print(URL)
    response = requests.get(URL)
    sumario_HTML = html.fromstring(response.text)

    documentos_Name = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/text()[1]')
    documentos_URL = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/@href')

    for row in range(len(documentos_Name)):
        DOUE_sumarios = DOUE_sumarios.append({'Seccion': 'L' + documentos_URL[row][-7:-4],
                                        'Item_Title': documentos_Name[row],
                                        'Item_Link': 'https://eur-lex.europa.eu' + documentos_URL[row][10:],
                                        #'Item_Tag': documentos_URL[row][-45:-29],
                                        'Item_Doc_Name': '',
                                        'Fecha_publicacion': '',
                                        'PDF_Link': '',
                                        'Tags': '',
                                        'Match_ASECORP_BBDD': ''},
                                        ignore_index=True)


In [195]:
# documentos_URL[1][-45:-29]

In [196]:
DOUE_sumarios

,Fecha_publicacion,Item_Doc_Name,Item_Link,Item_Title,Match_ASECORP_BBDD,PDF_Link,Seccion,Tags
0,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento Delegado (UE) 2021/277 de la Comisi...,,,L062,
1,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento de Ejecución (UE) 2021/278 de la Co...,,,L062,
2,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento de Ejecución (UE) 2021/279 de la Co...,,,L062,
3,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento de Ejecución (UE) 2021/280 de la Co...,,,L062,
4,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (Euratom) 2021/281 del Consejo, de 22...",,,L062,
5,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/282 del Consejo, de 22 de...",,,L062,
6,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/283 del Consejo, de 22 de...",,,L062,
7,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/284 del Consejo, de 22 de...",,,L062,
8,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/285 del Consejo, de 22 de...",,,L062,
9,,,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/286 del Consejo, de 22 de...",,,L062,


In [197]:
print(DOUE_sumarios['Item_Link'][1])

https://eur-lex.europa.eu/legal-content/ES/AUTO/?uri=uriserv:OJ.L_.2021.062.01.0004.01.SPA&toc=OJ:L:2021:062:TOC


In [198]:
def save_txt(txt, path):
    with open(path, 'w') as f:
        f.write(txt)


In [199]:
documentos_FileName = sumario_HTML.xpath('//*[@class="DocumentTitle pull-left"]/text()')

In [200]:
row = 0
for link in DOUE_sumarios['Item_Link']:
    sumario_Text = []
    ### Recoge PDF link en Items Legislación

    response = requests.get(link)
    sumario_HTML = html.fromstring(response.text)

    PDF_Link = sumario_HTML.xpath('//*[@id="format_language_table_PDF_ES"]/@href')
    DOUE_sumarios['PDF_Link'][row] = 'https://eur-lex.europa.eu' + PDF_Link[0][10:]

    Item_Doc_Name = sumario_HTML.xpath('//*[@class="DocumentTitle pull-left"]/text()')[0][9:]
    DOUE_sumarios['Item_Doc_Name'][row] = Item_Doc_Name

    #Item_Fecha = str(sumario_HTML.xpath('//*[@class="oj-hd-date"]/text()'))
    #Item_Fecha = datetime.strptime(str(raiz.xpath('//*[@class="oj-hd-date"]/text()'))[0:-2],"%d.%m.%Y")
    Item_Fecha = re.findall('[0-9]+\.[0-9]+\.[0-9]+', str(sumario_HTML.xpath('//*[@class="oj-hd-date"]/text()')))
    DOUE_sumarios['Fecha_publicacion'][row] = datetime.strptime(str(Item_Fecha)[2:-2],"%d.%m.%Y").date()

    ### Recoge Texto de Items
    
    Item_Text = sumario_HTML.xpath('//*[@class="oj-normal"]/text()')
    
    #[ sumario_Text.append(re.sub('(\\r||\\n\\t|\\xa0)+', '', item)) for item in Item_Text ]
    [ sumario_Text.append(item) for item in Item_Text ]
   
    ### Sustituye caracteres 'n.' por 'n.º' ya que al importar de HTML se pierde el caracter º

    sumario = "".join(sumario_Text).replace(' n.',' n.º')
    print(sumario)
    ### Busca tags en texto

    DOUE_sumarios['Tags'][row] = list(set(re.findall('|'.join(pattern), sumario, flags=re.IGNORECASE)))

    ### Salva texto de Items en formato TXT

    save_txt(sumario, './DOUEs/Resumen-DOUE-' + Item_Doc_Name + '.txt')
    #print(sumario)
    #print('./DOUEs/Resumen-DOUE-' + Item_Doc_Name + '.txt')
    #print(Item_Doc_Name)

    #print(list(set(re.findall('|'.join(pattern), sumario, flags=re.IGNORECASE))))

    row += 1


LA COMISIÓN EUROPEA,Visto el Tratado de Funcionamiento de la Unión Europea,Visto el Reglamento (UE) 2019/1021 del Parlamento Europeo y del Consejo, de 20 de junio de 2019, sobre contaminantes orgánicos persistentes , y en particular su artículo 15, apartado 1,Considerando lo siguiente:(1)El Reglamento (UE) 2019/1021 transpone los compromisos de la Unión en virtud del Convenio de Estocolmo sobre contaminantes orgánicos persistentes  y del Protocolo del Convenio de 1979 sobre la contaminación atmosférica transfronteriza a gran distancia provocada por contaminantes orgánicos persistentes .(2)De conformidad con el artículo 3, apartado 1, del Reglamento (UE) 2019/1021, están prohibidas la fabricación, la comercialización y el uso de las sustancias enumeradas en el anexo I de dicho Reglamento, solas, en mezclas o en artículos, a reserva de lo dispuesto en el artículo 4 de dicho Reglamento.(3)El pentaclorofenol y sus sales y ésteres figuran en el anexo I del Reglamento (UE) 2019/1021 sin valo

In [201]:
# Elimina Tags duplicados
for i, row in DOUE_sumarios.iterrows():
    DOUE_sumarios['Tags'][i] = list(set(DOUE_sumarios['Tags'][i]))
    print(DOUE_sumarios['Tags'][i])

['Reglamento (UE) 2019/1021']
[]
['Reglamento (UE) 2017/625', 'Reglamento (UE) 2018/848']
['Reglamento (UE) 2017/1485', 'Reglamento (UE) 2019/943', 'Reglamento (UE) 2016/1719', 'Reglamento (UE) 2017/2195', 'Reglamento (UE) 2019/942', 'Reglamento (UE) 2015/1222']
['Reglamento (UE) 2017/1939']
[]
[]
[]
[]
[]
['Decisión de Ejecución (UE) 2019/2151']
['Decisión de Ejecución (UE) 2019/2151']


In [202]:
DOUE_sumarios

,Fecha_publicacion,Item_Doc_Name,Item_Link,Item_Title,Match_ASECORP_BBDD,PDF_Link,Seccion,Tags
0,2021-02-23,32021R0277,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento Delegado (UE) 2021/277 de la Comisi...,,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[Reglamento (UE) 2019/1021]
1,2021-02-23,32021R0278,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento de Ejecución (UE) 2021/278 de la Co...,,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[]
2,2021-02-23,32021R0279,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento de Ejecución (UE) 2021/279 de la Co...,,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,"[Reglamento (UE) 2017/625, Reglamento (UE) 201..."
3,2021-02-23,32021R0280,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento de Ejecución (UE) 2021/280 de la Co...,,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,"[Reglamento (UE) 2017/1485, Reglamento (UE) 20..."
4,2021-02-23,32021D0281,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (Euratom) 2021/281 del Consejo, de 22...",,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[Reglamento (UE) 2017/1939]
5,2021-02-23,32021D0282,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/282 del Consejo, de 22 de...",,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[]
6,2021-02-23,32021D0283,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/283 del Consejo, de 22 de...",,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[]
7,2021-02-23,32021D0284,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/284 del Consejo, de 22 de...",,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[]
8,2021-02-23,32021D0285,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/285 del Consejo, de 22 de...",,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[]
9,2021-02-23,32021D0286,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Decisión (PESC) 2021/286 del Consejo, de 22 de...",,https://eur-lex.europa.eu/legal-content/ES/TXT...,L062,[]


In [203]:
### Ordena Columnas
DOUE_sumarios = DOUE_sumarios[['Seccion', 'Item_Doc_Name', 'Fecha_publicacion', 'PDF_Link', 'Item_Title', 'Item_Link', 'Tags', 'Match_ASECORP_BBDD']]
DOUE_sumarios

,Seccion,Item_Doc_Name,Fecha_publicacion,PDF_Link,Item_Title,Item_Link,Tags,Match_ASECORP_BBDD
0,L062,32021R0277,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento Delegado (UE) 2021/277 de la Comisi...,https://eur-lex.europa.eu/legal-content/ES/AUT...,[Reglamento (UE) 2019/1021],
1,L062,32021R0278,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento de Ejecución (UE) 2021/278 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
2,L062,32021R0279,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento de Ejecución (UE) 2021/279 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,"[Reglamento (UE) 2017/625, Reglamento (UE) 201...",
3,L062,32021R0280,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento de Ejecución (UE) 2021/280 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,"[Reglamento (UE) 2017/1485, Reglamento (UE) 20...",
4,L062,32021D0281,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (Euratom) 2021/281 del Consejo, de 22...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[Reglamento (UE) 2017/1939],
5,L062,32021D0282,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/282 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
6,L062,32021D0283,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/283 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
7,L062,32021D0284,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/284 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
8,L062,32021D0285,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/285 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
9,L062,32021D0286,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/286 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],


In [204]:
### Crea lista plana de Tags sin duplicados
DOUE_flat_list = list(set([item for row in DOUE_sumarios['Tags'] for item in row ]))

In [205]:
print (DOUE_flat_list)

['Reglamento (UE) 2018/848', 'Reglamento (UE) 2017/1485', 'Reglamento (UE) 2019/943', 'Reglamento (UE) 2017/1939', 'Decisión de Ejecución (UE) 2019/2151', 'Reglamento (UE) 2019/1021', 'Reglamento (UE) 2016/1719', 'Reglamento (UE) 2017/2195', 'Reglamento (UE) 2019/942', 'Reglamento (UE) 2017/625', 'Reglamento (UE) 2015/1222']


In [206]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
list(set(DOUE_flat_list) & set(boletin_ASECORP_flat_list))

[]

In [207]:
for i, row_to_compare in DOUE_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOUE_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

In [208]:
DOUE_sumarios

,Seccion,Item_Doc_Name,Fecha_publicacion,PDF_Link,Item_Title,Item_Link,Tags,Match_ASECORP_BBDD
0,L062,32021R0277,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento Delegado (UE) 2021/277 de la Comisi...,https://eur-lex.europa.eu/legal-content/ES/AUT...,[Reglamento (UE) 2019/1021],
1,L062,32021R0278,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento de Ejecución (UE) 2021/278 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
2,L062,32021R0279,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento de Ejecución (UE) 2021/279 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,"[Reglamento (UE) 2017/625, Reglamento (UE) 201...",
3,L062,32021R0280,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,Reglamento de Ejecución (UE) 2021/280 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,"[Reglamento (UE) 2017/1485, Reglamento (UE) 20...",
4,L062,32021D0281,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (Euratom) 2021/281 del Consejo, de 22...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[Reglamento (UE) 2017/1939],
5,L062,32021D0282,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/282 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
6,L062,32021D0283,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/283 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
7,L062,32021D0284,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/284 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
8,L062,32021D0285,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/285 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],
9,L062,32021D0286,2021-02-23,https://eur-lex.europa.eu/legal-content/ES/TXT...,"Decisión (PESC) 2021/286 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,[],


In [209]:
## Aplica función de conversión de listas a strings
DOUE_sumarios = DOUE_sumarios.apply(lambda x: [list2Str(i) for i in x])

In [210]:
DOUE_sumarios_CSV = pd.DataFrame()
DOUE_sumarios_CSV = DOUE_sumarios

In [211]:
# Compone y genera enlace a PDF del DOUE correspondiente
DOUE_sumarios_CSV['PDF_Link'] = '=HIPERVINCULO(' + '"' + DOUE_sumarios['PDF_Link'] + '";' + '"' + "Doc: " + DOUE_sumarios['Item_Doc_Name'] + '")'

In [212]:
DOUE_sumarios_CSV

,Seccion,Item_Doc_Name,Fecha_publicacion,PDF_Link,Item_Title,Item_Link,Tags,Match_ASECORP_BBDD
0,L062,32021R0277,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...",Reglamento Delegado (UE) 2021/277 de la Comisi...,https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento (UE) 2019/1021,
1,L062,32021R0278,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...",Reglamento de Ejecución (UE) 2021/278 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,,
2,L062,32021R0279,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...",Reglamento de Ejecución (UE) 2021/279 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Reglamento (UE) 2017/625, Reglamento (UE) 2018...",
3,L062,32021R0280,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...",Reglamento de Ejecución (UE) 2021/280 de la Co...,https://eur-lex.europa.eu/legal-content/ES/AUT...,"Reglamento (UE) 2017/1485, Reglamento (UE) 201...",
4,L062,32021D0281,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...","Decisión (Euratom) 2021/281 del Consejo, de 22...",https://eur-lex.europa.eu/legal-content/ES/AUT...,Reglamento (UE) 2017/1939,
5,L062,32021D0282,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...","Decisión (PESC) 2021/282 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,,
6,L062,32021D0283,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...","Decisión (PESC) 2021/283 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,,
7,L062,32021D0284,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...","Decisión (PESC) 2021/284 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,,
8,L062,32021D0285,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...","Decisión (PESC) 2021/285 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,,
9,L062,32021D0286,2021-02-23,"=HIPERVINCULO(""https://eur-lex.europa.eu/legal...","Decisión (PESC) 2021/286 del Consejo, de 22 de...",https://eur-lex.europa.eu/legal-content/ES/AUT...,,


In [213]:
DOUE_sumarios_CSV['PDF_Link'][1]

'=HIPERVINCULO("https://eur-lex.europa.eu/legal-content/ES/TXT/PDF/?uri=CELEX:32021R0278&from=ES";"Doc: 32021R0278")'

In [214]:
### Genera fichero CSV

DOUE_sumarios_CSV.to_csv("./ASECORP/Resultados_Matching_DOUE_" + today.strftime("%Y%m%d") + ".csv", index=False) 